# 소프트맥스 함수
$$ e^z_i \over {e^z_1+e^z_2+e^z_3} $$
$$ z=-ln({1 \over \hat{y}} -1) $$

# 크로스 엔트로피 손실 함수
$$ L=-log(a_{y=1}) $$
$$ {\partial L \over \partial z} = {-(y-a)} $$

In [1]:
# 다중 분류 신경망을 구현합니다.
class MinibatchNetwork:
    def __init__(self, units=10, batch_size=32, learning_rate=0.1, l1=0, l2=0):
        super().__init__(units,learning_rate,l1,l2)
        self.batch_size=batch_size    # 배치 크기
    def fit(self, x, y, epochs=100, x_val=None, y_val=None):
        y_val=y_val.reshape(-1,1)     # 타깃을 열 벡터로 바꿉니다.
        self.init_weights(x.shape[1]) # 은닉층과 출력층의 가중치를 초기화합니다.
        np.random.seed(42)
        # epochs만큼 반복합니다.
        for i in range(epochs):
            loss=0
            # 제너레이터 함수에서 반환한 미니 배치를 순화합니다.
            for x_batch, y_batch in self.gen_batch(x,y):
                y_batch=y_batch.reshape(-1,1)  # 타깃을 열 벡터로 바꿉니다.
                m=len(x_batch)                 # 샘플 개수를 저장합니다.
                a=self.training(x_batch, y_batch, m)
                # 안전한 로그 계산을 위해 클리핑합니다.
                a=np.clip(a, 1e-10, 1-1e-10)
                # 로그 손실과 규제 손실을 더하여 리스트에 추가합니다.
                loss+=np.sum(-(y_batch*np.log(a)+(1-y_batch)*np.log(1-a)))
            self.losses.append((loss+self.reg_loss())/len(x))
            # 검증 세트에 대한 손실을 계산합니다.
            self.update_val_loss(x_val, y_val)
    # 미니 배치 제너레이터 함수
    def gen_batch(self, x,y):
        length=len(x)
        bins=length//self.batch_size # 미니 배치 횟수
        if length%self.batch_size:
            bins+=1                  # 나누어 떨어지지 않을 때
        indexes=np.random.permutation(np.arange(len(x))) # 인덱스를 섞습니다.
        x=x[indexes]
        y=y[indexes]
        for i in range(bins):
            start=self.batch_size*i
            end=self.batch_size*(i+1)
            yield x[start:end], y[start:end] # 배치 사이즈만큼 슬라이싱하여 반환합니다.
    def sigmoid(self,z):
        a=1/(1+np.exp(-z)) # 시그모이드 계산
        return a
    def softmax(self,z):
        # 소프트맥스 함수
        exp_z=np.exp(z)
        return exp_z/np.sum(exp_z,axis=1).reshape(-1,1)
    def forpass(self,x):
        z1=np.dot(x, self.w1)+self.b1       # 첫 번째 층의 선형식을 계산합니다.
        self.a1=self.sigmoid(z1)            # 활성화 함수를 적용합니다.
        z2=np.dot(self.a1, self.w2)+self.b2 # 두 번째 층의 선형식을 계산합니다.
    def init_weights(self, n_features):
        np.random.seed(42)
        self.w1=np.random.normal(0,1,(n_features, self.units)) # (특성 개수, 은닉층의 크기)
        self.b1=np.zeros(self.units)                           # 은닉층의 크기
        self.w2=np.random.normal(0,1,(self.units,n_classes))           # (은닉층의 크기,1)
        self.b2=np.zeros(n_classes)
    def fit(self,x,y,epochs=100, x_val=None, y_val=None):
        np.random.seed(42)
        self.init_weights(x.shape[1],y.shape[1]) # 은닉층과 출력층의 가중치를 초기화합니다.
        # epochs만큼 반복합니다.
        for i in range(epochs):
            loss=0
            print(',',end='')
            # 제너레이터 함수에서 반환한 미니 배치를 순환합니다.
            for x_batch, y_batch in self.gen_batch(x,y):
                a=self.training(x_batch,y_batch)
                # 안전한 로그 계산을 위해 클리핑합니다.
                a=np.clip(a,1e-10,1-1e-10)
                # 로그 손실과 규제 손실을 더하여 리스트에 추가합니다.
                loss+=np.sum(-y_batch*np.log(a))
            self.losses.append((loss+self.reg_loss)/len(x))
            # 검증 세트에 대한 손실을 계산합니다.
            self.update_val_loss(x_val, y_val)
    def trainig(self, x, y):
        m=len(x)           # 샘플 개수를 저장합니다.
        z=self.forpass(x)  # 정방향 계산을 수행합니다.
        a=self.softmax(z)  # 활성화 함수를 적용합니다.
        err=-(y-a)             # 오차를 계산합니다.
        # 오차를 역전파하여 그레디언트를 계산합니다.
        w1_grad, b1_grad, w2_grad, b2_grad=self.backprop(x,err)
        # 그레디언트에서 페널티 항의 미분값을 뺍니다.
        w1_grad+=(self.l1*np.sign(self.w1)+self.l2*self.w1)/m
        w2_grad+=(self.l1*np.sign(self.w2)+self.l2*self.w2)/m
        # 은닉층의 가중치와 절편을 업데이트합니다.
        self.w1-=self.lr*w1_grad
        self.b1-=self.lr*b1_grad
        # 출력층의 가중치와 절편을 업데이트합니다.
        self.w2-=self.lr*w2_grad
        self.b2-=self.lr*b2_grad
        return a
    def predict(self,x):
        z=self.forpass(x)           # 정방향 계산을 수행합니다.
        return np.argmax(z, axis=1)# 가장 큰 값의 인덱스를 반환합니다.
    def score(self, x, y):
        # 예측과 타깃 열 벡터를 비교하여 True의 비율을 반환합니다.
        return np.mean(self.predict(x)==np.argmax(y,axis=1))
    def update_val_loss(self, x_val,y_val):
        z=self.forpass(x_val)      #정방향 계산을 수행합니다.
        a=self.softmax(z)          # 활성화 함수를 적용합니다.
        a=np.clip(a,1e-10,1-1e-10) # 출력값을 클리핑합니다.
        # 크로스 엔트로피 손실과 규제 손실을 더하여 리스트에 추가합니다.
        val_loss=np.sum(-y_val*np.log(a))
        self.val_losses.append((val_loss+self.reg_loss())/len(y_val))

In [ ]:
# 의류 이미지를 분류합니다.
!pip install tensorflow_gpu==2.2.0

In [2]:
# 의류 데이터를 준비합니다.
import tensorflow as tf
tf.__version__

ImportError: Traceback (most recent call last):
  File "C:\Users\82105\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 64, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: DLL 초기화 루틴을 실행할 수 없습니다.


Failed to load the native TensorFlow runtime.

See https://www.tensorflow.org/install/errors

for some common reasons and solutions.  Include the entire stack trace
above this error message when asking for help.

In [3]:
(x_train_all, y_train_all),(x_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()

NameError: name 'tf' is not defined